## Helmet Detection With COCO {with.pt} and With CBAM


#### Importing the Modules

## Integrating The CBAM

In [1]:
import torch
import torch.nn as nn

class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.fc = nn.Sequential(
            nn.Conv2d(channels, channels // reduction, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(channels // reduction, channels, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return x * self.sigmoid(
            self.fc(self.avg_pool(x)) + self.fc(self.max_pool(x))
        )


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg = torch.mean(x, dim=1, keepdim=True)
        max_, _ = torch.max(x, dim=1, keepdim=True)
        return self.sigmoid(self.conv(torch.cat([avg, max_], dim=1)))


class CBAM(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.ca = ChannelAttention(channels)
        self.sa = SpatialAttention()

    def forward(self, x):
        x = self.ca(x)
        return x * self.sa(x)


## Registerng the CBAM


In [ ]:
from ultralytics.nn.modules import __dict__ as ULTRALYTICS_MODULES

ULTRALYTICS_MODULES["CBAM"] = CBAM


In [4]:
import ultralytics.nn.modules as modules

# register CBAM globally so YAML parser can resolve it
setattr(modules, "CBAM", CBAM)

# ALSO register inside all known submodules (safe even if not used)
import ultralytics.nn.modules.block as block
import ultralytics.nn.modules.conv as conv
import ultralytics.nn.modules.head as head

setattr(block, "CBAM", CBAM)
setattr(conv, "CBAM", CBAM)
setattr(head, "CBAM", CBAM)


In [5]:
import ultralytics.nn.modules as modules
print("modules:", hasattr(modules, "CBAM"))

import ultralytics.nn.modules.block as block
print("block  :", hasattr(block, "CBAM"))


modules: True
block  : True


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n_cbam.yaml")

model.train(
    data=r"C:/Users/soumy/OneDrive/Desktop/Helmet Detection/Helmet-Detection/Helmet-Detection/dataset/data1.yaml",
    epochs=100,
    imgsz=640,
    batch=8,          
    device=0,
    workers=0,
    amp=True,
    cos_lr=True,
    close_mosaic=10,
    name="helmet_yolov8n_cbam"
)


KeyError: 'CBAM'

In [6]:
from ultralytics import YOLO
model = YOLO("yolov8n_cbam.yaml")


KeyError: 'CBAM'